<a href="https://colab.research.google.com/github/arvindnswamy/RemoteSensing/blob/development/SupervisedClassificationWithPolygons_GEEtutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
from IPython.display import Image

# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Xof2jos-wsRvs4zOkHDN0CN6rjU7fXpFRh5ACsv9ZQo&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AGzv1oSkICusYyDpi3rWHu59gesFFgXMCz8EaubaeC62C4aWu7bEkM

Successfully saved authorization token.


In [2]:
ee.Initialize()

def get_images(path_list, row_list, satellite, start_date, end_date, max_cloud_percentage):
  coll = ee.ImageCollection(satellite).filterDate(start_date, end_date)\
            .filter(ee.Filter.inList('WRS_PATH', path_list))\
            .filter(ee.Filter.inList('WRS_ROW', row_list))\
            .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_percentage))
  image_ids = list(map(lambda x: x['id'], coll.getInfo()['features']))

  images = list(map(lambda x: ee.Image(x), image_ids))

  return images

In [5]:
'''
Functions needed to display images and features/featurecollections are given here. 
These functions are all from other sources. Mostly from google tutorials.
'''
!pip install geojson
!pip install pygeoj
import geojson
import json
import pygeoj
import numpy as np


import folium


# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

  Created wheel for pygeoj: filename=PyGeoj-1.0.0-cp36-none-any.whl size=10348 sha256=5c4bbacbda3638b7033c7152cc10439ccfa93af475e0bde11eb94f187eb67356
  Stored in directory: /root/.cache/pip/wheels/7d/7c/8e/3c82fc15ee9f04d8df3e59817d2ee5499a8413341d6476c7d8
Successfully built pygeoj


In [6]:
#Python version of GEE tutorial available at https://developers.google.com/earth-engine/classification
#This is the example that uses libsvm classifier.

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1');

imageEx = ee.Algorithms.Landsat.simpleComposite(collection = l8.filterDate('2018-03-01', '2018-04-30'), asFloat = True);


# Manually created polygons.
forest1 = ee.Geometry.Rectangle(-63.0187, -9.3958, -62.9793, -9.3443);
forest2 = ee.Geometry.Rectangle(-62.8145, -9.206, -62.7688, -9.1735);
nonForest1 = ee.Geometry.Rectangle(-62.8161, -9.5001, -62.7921, -9.4486);
nonForest2 = ee.Geometry.Rectangle(-62.6788, -9.044, -62.6459, -8.9986);

# Make a FeatureCollection from the hand-made geometries.
polygons = ee.FeatureCollection([
  ee.Feature(nonForest1, {'class': 0}),
  ee.Feature(nonForest2, {'class': 0}),
  ee.Feature(forest1, {'class': 1}),
  ee.Feature(forest2, {'class': 1}),
]);

trainingEx = imageEx.sampleRegions(collection =  polygons, properties = ['class'], scale = 30.0)

In [11]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

classifierEx = ee.Classifier.libsvm(kernelType = 'RBF', gamma = 0.5, cost = 10)
trainedEx = classifierEx.train(trainingEx, 'class', bands);

#trainedEx = ee.Classifier.smileCart().train(trainingEx, 'class', bands)
classifiedEx = imageEx.classify(trainedEx);

'''
from IPython.display import Image
parameters = {'min': 0.0,
              'max': 2.0,
              'dimensions': 400,
              'palette': ['red', 'green', 'blue'],
              #'region': ee.Geometry.Rectangle([-74.04, 40.69, -73.82, 40.94])
}
Image(url = classifiedEx.getThumbUrl(parameters))
'''

# What properties do the constituent features of the featureCollection have?
#nycFC.getInfo()['features'][0]['properties']
#print(polygons.getInfo())
#fc.getInfo()

my_map = folium.Map(location=[-9.2399, -62.836], zoom_start=9, height=500)
my_map.add_ee_layer(classifiedEx, {'min': 0, 'max': 1, 'palette': ['red', 'green']}, 'water')
display(my_map)

In [12]:
type(imageEx)

ee.image.Image

In [13]:
type(classifiedEx)

ee.image.Image

In [14]:
trainingEx.toList(100).getInfo()

[{'geometry': None,
  'id': '0_0',
  'properties': {'B1': 0.16775566339492798,
   'B10': 295.8961181640625,
   'B11': 291.8773193359375,
   'B2': 0.14853456616401672,
   'B3': 0.13416661322116852,
   'B4': 0.1338568478822708,
   'B5': 0.31142058968544006,
   'B6': 0.27176958322525024,
   'B7': 0.1475754976272583,
   'B8': 0.1491023600101471,
   'B9': 0.0033248187974095345,
   'class': 0},
  'type': 'Feature'},
 {'geometry': None,
  'id': '0_1',
  'properties': {'B1': 0.15361493825912476,
   'B10': 290.6630554199219,
   'B11': 287.24853515625,
   'B2': 0.12374471127986908,
   'B3': 0.09260939806699753,
   'B4': 0.06091181933879852,
   'B5': 0.30116212368011475,
   'B6': 0.09558470547199249,
   'B7': 0.039030417799949646,
   'B8': 0.10031708329916,
   'B9': 0.00522435549646616,
   'class': 0},
  'type': 'Feature'},
 {'geometry': None,
  'id': '0_2',
  'properties': {'B1': 0.15026478469371796,
   'B10': 290.7875671386719,
   'B11': 287.4181213378906,
   'B2': 0.12030085176229477,
   'B3':

In [15]:
classifiedEx.toArray().getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'dimensions': 1,
    'max': 2147483647,
    'min': -2147483648,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'array'}],
 'type': 'Image'}